# **Query Chembl for Approved Small Molecule Drugs**
I wanted to document some of my learnings by building a SQL query that retrives the chemical structures and data from the Chembl database. Prior to running, I already downloaded the Chembl database and hosted it locally using the Postgres App for Mac.

Instructions for downloading the Chembl database can be found [here](https://chembl.gitbook.io/chembl-interface-documentation/downloads) and installation instructions for Postgres App [here](https://postgresapp.com).

### **1. Import modules**

I prefer working with the pandas and adbc_driver_postgresql modules for Postgres queries and dataframes. By default, pandas truncates the number of columns printed. I adjusted the max column setting to print all columns.

In [1]:
# Import modules
import pandas as pd
import adbc_driver_postgresql
import adbc_driver_postgresql.dbapi

# Expand to see all columns
pd.set_option('display.max_columns', None)

# Print versions
print(f"Pandas Version: {pd.__version__}")
print(f"ADBC Driver Version: {adbc_driver_postgresql.__version__}")

Pandas Version: 2.2.3
ADBC Driver Version: 1.5.0


### 2. **Test Connection to PostgreSQL**

I set the URI hosting the Chembl database on my localhost. I used a very basic SQL query to test the connection to Postgres.

In [2]:
uri = "postgresql://localhost/Chembl35"
try:
    conn = adbc_driver_postgresql.dbapi.connect(uri)
    with conn.cursor() as cur:
        cur.execute("SELECT 1")
        assert cur.fetchone() == (1,)
    print("Database connected successfully")
except:
    print("Database not connected successfully")

Database connected successfully


### 3. **Preview All Available Tables in Chembl35 Database**

The Chembl database contains a lot of data which has been split into separate tables. I often need to query and merge multiple tables to get all the data needed. I ran this command to list all the available tables in Chembl.

In [3]:
sql_list_tables = """
SELECT table_name
FROM information_schema.tables
    WHERE table_schema = 'public'
ORDER BY table_name
"""
with conn.cursor() as cur:
    cur.execute(sql_list_tables)
    chembl_tables_df = pd.DataFrame(
        cur.fetchall(),
        columns=[desc[0] for desc in cur.description]
    )
print(chembl_tables_df["table_name"].values)

['action_type' 'activities' 'activity_properties' 'activity_smid'
 'activity_stds_lookup' 'activity_supp' 'activity_supp_map'
 'assay_class_map' 'assay_classification' 'assay_parameters' 'assay_type'
 'assays' 'atc_classification' 'binding_sites' 'bio_component_sequences'
 'bioassay_ontology' 'biotherapeutic_components' 'biotherapeutics'
 'cell_dictionary' 'chembl_id_lookup' 'chembl_release' 'component_class'
 'component_domains' 'component_go' 'component_sequences'
 'component_synonyms' 'compound_properties' 'compound_records'
 'compound_structural_alerts' 'compound_structures'
 'confidence_score_lookup' 'curation_lookup' 'data_validity_lookup'
 'defined_daily_dose' 'docs' 'domains' 'drug_indication' 'drug_mechanism'
 'drug_warning' 'formulations' 'frac_classification' 'go_classification'
 'hrac_classification' 'indication_refs' 'irac_classification'
 'ligand_eff' 'mechanism_refs' 'metabolism' 'metabolism_refs'
 'molecule_atc_classification' 'molecule_dictionary'
 'molecule_frac_class

### 4. **Preview All Columns in Compound Structures Table**

To see which columns are available in the compound structures table, I ran this query. Canonical SMILES are available in this table as well.

In [4]:
sql_table = "compound_structures"
sql_list_columns = """
SELECT column_name
FROM information_schema.columns
    WHERE table_name = 'compound_records'
ORDER BY ordinal_position
"""
with conn.cursor() as cur:
    cur.execute(sql_list_columns)
    chembl_columns_df = pd.DataFrame(
        cur.fetchall(),
        columns=[desc[0] for desc in cur.description]
    )
print(f"Table name: {sql_table}")
print(f"Columns: {chembl_columns_df["column_name"].values}")

Table name: compound_structures
Columns: ['record_id' 'molregno' 'doc_id' 'compound_key' 'compound_name' 'src_id'
 'src_compound_id' 'cidx']


### 5. **Preview All Columns in Molecule Dictionary**

To see which columns are available in the molecule dictionary table, I ran this query. The molecule dictionary table contains much of the data about the drug information.

In [5]:
sql_table = "molecule_dictionary"
sql_list_columns = """
SELECT column_name
FROM information_schema.columns
    WHERE table_name = 'molecule_dictionary'
ORDER BY ordinal_position
"""
with conn.cursor() as cur:
    cur.execute(sql_list_columns)
    chembl_columns_df = pd.DataFrame(
        cur.fetchall(),
        columns=[desc[0] for desc in cur.description]
    )
print(f"Table name: {sql_table}")
print(f"Columns: {chembl_columns_df["column_name"].values}")

Table name: molecule_dictionary
Columns: ['molregno' 'pref_name' 'chembl_id' 'max_phase' 'therapeutic_flag'
 'dosed_ingredient' 'structure_type' 'chebi_par_id' 'molecule_type'
 'first_approval' 'oral' 'parenteral' 'topical' 'black_box_warning'
 'natural_product' 'first_in_class' 'chirality' 'prodrug' 'inorganic_flag'
 'usan_year' 'availability_type' 'usan_stem' 'polymer_flag' 'usan_substem'
 'usan_stem_definition' 'indication_class' 'withdrawn_flag'
 'chemical_probe' 'orphan']


### 6. **Preview All Columns in Compound Properties**

To see which columns are available in the compound properties table, I ran this query. Compound properties are mostly physicochemical properties of the molecule.

In [6]:
sql_table = "compound_properties"
sql_list_columns = """
SELECT column_name
FROM information_schema.columns
    WHERE table_name = 'compound_properties'
ORDER BY ordinal_position
"""
with conn.cursor() as cur:
    cur.execute(sql_list_columns)
    chembl_columns_df = pd.DataFrame(
        cur.fetchall(),
        columns=[desc[0] for desc in cur.description]
    )
print(f"Table name: {sql_table}")
print(f"Columns: {chembl_columns_df["column_name"].values}")

Table name: compound_properties
Columns: ['molregno' 'mw_freebase' 'alogp' 'hba' 'hbd' 'psa' 'rtb' 'ro3_pass'
 'num_ro5_violations' 'cx_most_apka' 'cx_most_bpka' 'cx_logp' 'cx_logd'
 'molecular_species' 'full_mwt' 'aromatic_rings' 'heavy_atoms'
 'qed_weighted' 'mw_monoisotopic' 'full_molformula' 'hba_lipinski'
 'hbd_lipinski' 'num_lipinski_ro5_violations' 'np_likeness_score']


### 7. **Query Chembl Database for Small Molecule Drugs and Merge Tables**

I took the information about the columns above and combined it into a query/filter for small molecule drugs. It may be little excessive to include all columns, but this is how you could keep all the data. I often find it easier to filter data later on, rather than go back to retreive or generate the data again.

In [7]:
sql_query="""
SELECT DISTINCT m.chembl_id AS molecule_chembl_id,
s.canonical_smiles AS smiles,
-- Molecule Dictionary
m.molregno, m.pref_name, m.max_phase, m.therapeutic_flag, m.dosed_ingredient,
m.structure_type, m.chebi_par_id, m.molecule_type, m.first_approval, m.oral,
m.parenteral, m.topical, m.black_box_warning, m.natural_product, m.first_in_class,
m.chirality, m.prodrug, m.inorganic_flag, m.usan_year, m.availability_type,
m.usan_stem, m.polymer_flag, m.usan_substem, m.usan_stem_definition,
m.indication_class, m.withdrawn_flag, m.chemical_probe, m.orphan,
-- Compound Properties
p.mw_freebase, p.alogp, p.hba, p.hbd, p.psa, p.rtb, p.ro3_pass,
p.num_ro5_violations, p.cx_most_apka, p.cx_most_bpka, p.cx_logp,
p.cx_logd, p.molecular_species, p.full_mwt, p.aromatic_rings,
p.heavy_atoms, p.qed_weighted, p.mw_monoisotopic, p.full_molformula,
p.hba_lipinski, p.hbd_lipinski, p.num_lipinski_ro5_violations,
p.np_likeness_score
-- Join tables
FROM compound_structures s
    RIGHT JOIN compound_properties p ON s.molregno = p.molregno
    RIGHT JOIN molecule_dictionary m ON p.molregno = m.molregno
    JOIN compound_records r ON m.molregno = r.molregno
    AND m.max_phase = 4
    AND m.molecule_type = 'Small molecule'
    AND m.inorganic_flag = 0
ORDER BY m.first_approval ASC
"""
with conn.cursor() as cur:
    cur.execute(sql_query)
    chembl_df = pd.DataFrame(
        cur.fetchall(),
        columns=[desc[0] for desc in cur.description]
    )
print(chembl_df.shape)
chembl_df.head()

(3517, 53)


,molecule_chembl_id,smiles,molregno,pref_name,max_phase,therapeutic_flag,dosed_ingredient,structure_type,chebi_par_id,molecule_type,first_approval,oral,parenteral,topical,black_box_warning,natural_product,first_in_class,chirality,prodrug,inorganic_flag,usan_year,availability_type,usan_stem,polymer_flag,usan_substem,usan_stem_definition,indication_class,withdrawn_flag,chemical_probe,orphan,mw_freebase,alogp,hba,hbd,psa,rtb,ro3_pass,num_ro5_violations,cx_most_apka,cx_most_bpka,cx_logp,cx_logd,molecular_species,full_mwt,aromatic_rings,heavy_atoms,qed_weighted,mw_monoisotopic,full_molformula,hba_lipinski,hbd_lipinski,num_lipinski_ro5_violations,np_likeness_score
0,CHEMBL1200542,CC(=O)OCC(=O)[C@H]1CC[C@H]2[C@@H]3CCC4=CC(=O)C...,674493,DESOXYCORTICOSTERONE ACETATE,4.0,1,1,MOL,34671.0,Small molecule,1939.0,0,1,0,0,1,0,1,0,0,NaN,0.0,-cort-; -ster-; -terone,0,-cort-; -ster-; -terone,"cortisone derivatives; steroids (androgens, an...",Adrenocortical Steroid (salt-regulating),0,0,0,372.51,4.27,4.0,0.0,60.44,3.0,N,0.0,None,None,3.77,3.77,NEUTRAL,372.51,0.0,27.0,0.69,372.2301,C23H32O4,4.0,0.0,0.0,1.96
1,CHEMBL1200728,Cl.N=C(N)N,674679,GUANIDINE HYDROCHLORIDE,4.0,1,1,MOL,32735.0,Small molecule,1939.0,1,0,0,0,0,0,2,0,0,NaN,1.0,guan-,0,guan-,antihypertensives (guanidine derivatives),None,0,0,0,59.07,-1.16,1.0,3.0,75.89,0.0,N,0.0,None,12.55,-1.24,-3.65,BASE,95.53,0.0,4.0,0.24,59.0483,CH6ClN3,3.0,5.0,0.0,0.32
2,CHEMBL1200982,CCC(C)C1(CC)C(=O)[N-]C(=O)NC1=O.[Na+],674933,BUTABARBITAL SODIUM,4.0,1,1,MOL,NaN,Small molecule,1939.0,1,0,0,0,0,0,0,0,0,NaN,1.0,-barb-,0,-barb-,barbituric acid derivatives,Sedative-Hypnotic,0,0,0,212.25,0.79,3.0,2.0,75.27,3.0,N,0.0,7.48,None,1.45,1.19,NEUTRAL,234.23,0.0,15.0,0.68,212.1161,C10H15N2NaO3,5.0,2.0,0.0,0.32
3,CHEMBL3989520,NCCc1c[nH]cn1.O=P(O)(O)O.O=P(O)(O)O,2197391,HISTAMINE PHOSPHATE,4.0,1,1,MOL,NaN,Small molecule,1939.0,0,1,0,1,0,0,2,0,0,NaN,0.0,None,0,None,None,None,0,0,0,111.15,-0.09,2.0,2.0,54.70,2.0,Y,0.0,None,9.58,-0.70,-2.85,BASE,307.14,1.0,8.0,0.56,111.0796,C5H15N3O8P2,3.0,3.0,0.0,0.00
4,CHEMBL449,CCC(C)C1(CC)C(=O)NC(=O)NC1=O,2393,BUTABARBITAL,4.0,1,0,MOL,3228.0,Small molecule,1939.0,1,0,0,0,1,0,0,0,0,NaN,1.0,-barb-,0,-barb-,barbituric acid derivatives,Sedative-Hypnotic,0,0,0,212.25,0.79,3.0,2.0,75.27,3.0,N,0.0,7.48,None,1.45,1.19,NEUTRAL,212.25,0.0,15.0,0.68,212.1161,C10H16N2O3,5.0,2.0,0.0,0.32
